# GymPy

Note:
Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint

# Import API key
from api_keys import g_key

In [23]:
# Part 1 data file
master_data_path = "Output/master_data.csv"
lat_lng_path1 = "Resources/australian_postcodes_loc.csv"
lat_lng_path2 = "Output/postcode_loc_df.csv"

# Read the metadata 
master_data_df = pd.read_csv(master_data_path)
lat_lng_df1 = pd.read_csv(lat_lng_path1)
lat_lng_df2 = pd.read_csv(lat_lng_path2)

In [24]:
master_data_df.shape

(2641, 21)

In [25]:
lat_lng_df1.shape

(18442, 8)

In [26]:
master_data_df.columns

Index(['Postcode', 'House', 'Semi-detached', 'Apartment', 'Other dwelling',
       'No bedrooms (studios, etc)', '1 bedroom', '2 bedrooms', '3 bedrooms',
       '4 bedrooms', '5 bedrooms', '6 or more bedrooms',
       'Couple with no children', 'Couple with children',
       'One parent with children', 'Other family', 'Sole person household',
       'Group household', 'People with Mental Health Conditions',
       'Total People', 'State'],
      dtype='object')

In [27]:
lat_lng_df1.columns

Index(['id', 'Postcodes', 'locality', 'state', 'lat', 'long', 'Lat_precise',
       'Long_precise'],
      dtype='object')

In [28]:
lat_lng_df2.columns

Index(['Unnamed: 0', 'Postcode', 'State', 'Latitude', 'Longitude',
       'Remoteness Area'],
      dtype='object')

In [29]:
master_data_df.dtypes

Postcode                                 int64
House                                    int64
Semi-detached                            int64
Apartment                                int64
Other dwelling                           int64
No bedrooms (studios, etc)               int64
1 bedroom                                int64
2 bedrooms                               int64
3 bedrooms                               int64
4 bedrooms                               int64
5 bedrooms                               int64
6 or more bedrooms                       int64
Couple with no children                  int64
Couple with children                     int64
One parent with children                 int64
Other family                             int64
Sole person household                    int64
Group household                          int64
People with Mental Health Conditions     int64
Total People                             int64
State                                   object
dtype: object

In [30]:
lat_lng_df1.dtypes

id                int64
Postcodes         int64
locality         object
state            object
lat             float64
long            float64
Lat_precise     float64
Long_precise    float64
dtype: object

In [31]:
lat_lng_df1 = lat_lng_df1.drop_duplicates(subset=["Postcodes"], keep='first')

In [32]:
lat_lng_df1.shape

(3167, 8)

In [33]:
master_data_with_coords_df = pd.merge(master_data_df, lat_lng_df1, how="left", left_on=["Postcode"], right_on=["Postcodes"])
master_data_with_coords_df = master_data_with_coords_df.drop("Postcodes",axis=1)
master_data_with_coords_df.head()                                           

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,People with Mental Health Conditions,Total People,State,id,locality,state,lat,long,Lat_precise,Long_precise
0,800,33,6,5876,4,172,1856,3749,2805,168,...,344,7149,7: Northern Territory,3.0,DARWIN,NT,-12.458684,130.836680,-12.393279,130.776661
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,1971,34330,7: Northern Territory,7.0,ALAWA,NT,-12.381806,130.866242,-12.380000,130.873000
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,1076,18634,7: Northern Territory,24.0,ANULA,NT,-12.391249,130.890470,-12.392800,130.890000
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,1112,19920,7: Northern Territory,38.0,BAGOT,NT,-12.428017,130.873315,-12.411487,130.855410
4,822,9415,655,168,348,216,873,3083,4392,1811,...,755,25304,7: Northern Territory,59.0,ACACIA HILLS,NT,-13.643283,136.947051,-12.754154,131.148940


In [34]:
master_data_with_coords_df = master_data_with_coords_df.drop('state', axis=1)
master_data_with_coords_df = master_data_with_coords_df.rename(columns={"State_x": "State"})

In [35]:
master_data_with_coords_df.shape

(2641, 27)

In [36]:
master_data_with_coords_df.head()

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Group household,People with Mental Health Conditions,Total People,State,id,locality,lat,long,Lat_precise,Long_precise
0,800,33,6,5876,4,172,1856,3749,2805,168,...,910,344,7149,7: Northern Territory,3.0,DARWIN,-12.458684,130.836680,-12.393279,130.776661
1,810,14828,2642,5649,85,207,1768,8552,9360,5484,...,1737,1971,34330,7: Northern Territory,7.0,ALAWA,-12.381806,130.866242,-12.380000,130.873000
2,812,9601,1127,1549,134,67,448,3389,6590,2353,...,592,1076,18634,7: Northern Territory,24.0,ANULA,-12.391249,130.890470,-12.392800,130.890000
3,820,4926,3337,6733,104,167,1463,7566,7580,2372,...,1843,1112,19920,7: Northern Territory,38.0,BAGOT,-12.428017,130.873315,-12.411487,130.855410
4,822,9415,655,168,348,216,873,3083,4392,1811,...,268,755,25304,7: Northern Territory,59.0,ACACIA HILLS,-13.643283,136.947051,-12.754154,131.148940


In [37]:
pop_greater_10000_coord_df = master_data_with_coords_df.loc[master_data_with_coords_df["Total People"] >= 10000]

In [38]:
pop_greater_10000_coord_df.shape

(852, 27)

In [39]:
wa_pop_greater_10000_df = pop_greater_10000_coord_df.loc[pop_greater_10000_coord_df["State"] == "5: Western Australia"]

In [40]:
wa_pop_greater_10000_df.shape

(90, 27)

In [56]:
wa_pop_greater_10000_df["Mental Health Ratio"]=wa_pop_greater_10000_df["People with Mental Health Conditions"] / wa_pop_greater_10000_df["Total People"]*100

C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [74]:
wa_pop_greater_10000_df.tail(20)

,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Total People,State,id,locality,lat,long,Lat_precise,Long_precise,Mental Health Ratio,Gyms
2254,6168,15165,3191,1998,358,71,1164,4764,9859,6259,...,24913,5: Western Australia,16070.0,COOLOONGUP,-32.281957,115.757730,-32.294585,115.762919,11.323405,
2255,6169,24877,764,1053,5,16,421,2153,9635,13319,...,35317,5: Western Australia,16078.0,SAFETY BAY,-32.338504,115.763949,-32.310068,115.731902,11.011694,
2256,6170,10800,268,208,0,4,129,554,3740,6325,...,17329,5: Western Australia,16082.0,LEDA,-32.265645,115.835972,-32.271548,115.801822,8.592533,
2257,6171,23684,823,89,187,24,340,1618,5719,15799,...,37697,5: Western Australia,16084.0,BALDIVIS,-32.328863,115.833257,-32.322218,115.828515,9.334960,
2258,6172,8418,545,0,0,6,43,513,1438,6438,...,13477,5: Western Australia,16085.0,PORT KENNEDY,-32.375470,115.753147,-32.376008,115.751602,10.551310,
2259,6173,7651,75,6,0,0,6,60,847,5973,...,12474,5: Western Australia,16086.0,SECRET HARBOUR,-32.407414,115.757908,-32.405125,115.763437,7.158890,
2267,6208,8954,521,24,234,35,306,1283,3180,4459,...,12946,5: Western Australia,16099.0,BLYTHEWOOD,-32.630616,115.847592,-32.675616,115.861876,10.613317,
2269,6210,48736,8899,1909,301,98,2154,9443,24183,25390,...,75034,5: Western Australia,16112.0,BARRAGUP,-32.557981,115.728286,-32.550200,115.796000,10.067436,
2285,6230,27955,5334,1100,229,62,1606,5154,15656,13168,...,44972,5: Western Australia,16191.0,BUNBURY,-33.341076,115.649205,-33.320997,115.638348,10.622165,
2286,6232,7518,591,26,67,3,69,862,2472,4656,...,11402,5: Western Australia,16205.0,EATON,-33.316625,115.704263,-33.316962,115.715853,10.164883,


In [43]:
# Store WA Postcode by WA greater than 10,000 population 
wa_pg10000_postcode = wa_pop_greater_10000_df["Postcode"].astype(int)


In [44]:
wa_pg10000_postcode

2140    6000
2142    6004
2146    6008
2147    6009
2148    6010
        ... 
2412    6450
2458    6530
2505    6714
2510    6722
2511    6725
Name: Postcode, Length: 90, dtype: int32

In [60]:
wa_pg10000_locations1 = wa_pop_greater_10000_df[["lat", "long"]].astype(float)

In [61]:
wa_pg10000_locations1

,lat,long
2140,-31.948762,115.859912
2142,-31.956931,115.874601
2146,-31.956599,115.811432
2147,-31.985791,115.804692
2148,-31.971647,115.776390
...,...,...
2412,-33.401152,122.211127
2458,-28.802491,114.862505
2505,-21.145063,116.349933
2510,-20.407853,118.606673


In [62]:
#wa_pg10000_locations = wa_pop_greater_10000_df[["Latitude", "Longitude"]].astype(float)

In [63]:
#wa_pg10000_locations

In [64]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [65]:
#Figure(layout=FigureLayo# Customise the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout)

In [69]:
# Convert Mental Health to float and store
# HINT: be sure to handle NaN values

loc_mental_health = wa_pop_greater_10000_df["Mental Health Ratio"].astype(float)


In [72]:

#Heatmap

#marker_layer = gmaps.symbol_layer(wa_pg10000_locations)
fig_mental_health = gmaps.figure(center = (-28, 132) ,zoom_level = 3.7)
mental_health_layer = gmaps.heatmap_layer(wa_pg10000_locations1, weights=loc_mental_health, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 0.5)
fig_mental_health.add_layer(mental_health_layer)
#fig_mental_health.add_layer(marker_layer)
fig_mental_health


Figure(layout=FigureLayout(height='420px'))

In [51]:
# set up additional columns to hold information
wa_pop_greater_10000_df['Gyms'] = ""
gyms_df = wa_pop_greater_10000_df.reset_index(drop=True)
gyms_df.head(20)

C:\Users\dalem\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Postcode,House,Semi-detached,Apartment,Other dwelling,"No bedrooms (studios, etc)",1 bedroom,2 bedrooms,3 bedrooms,4 bedrooms,...,Total People,State,id,locality,lat,long,Lat_precise,Long_precise,Mental Health Ratio,Gyms
0,6000,1053,1405,9627,8,312,5210,8786,2654,398,...,13812,5: Western Australia,15676.0,CITY DELIVERY CENTRE,-31.948762,115.859912,-31.955000,115.855000,8.746018,
1,6004,426,676,10005,0,376,3560,8050,3754,422,...,11681,5: Western Australia,15682.0,EAST PERTH,-31.956931,115.874601,-31.954011,115.878793,7.328140,
2,6008,6589,2439,4672,21,175,3143,5534,5604,2180,...,16131,5: Western Australia,15688.0,DAGLISH,-31.956599,115.811432,-31.951851,115.809734,8.753332,
3,6009,8449,1808,2274,0,94,1033,3000,4487,3701,...,18936,5: Western Australia,15692.0,BROADWAY NEDLANDS,-31.985791,115.804692,-31.983680,115.814505,7.013097,
4,6010,8189,2857,3099,6,13,1379,4430,5590,4326,...,18863,5: Western Australia,15697.0,CLAREMONT,-31.971647,115.776390,-31.984246,115.778000,7.130361,
5,6014,10138,2149,4393,0,58,1949,6223,5724,4579,...,22092,5: Western Australia,15705.0,FLOREAT,-31.936389,115.808169,-31.934427,115.790646,7.016114,
6,6016,4789,1832,1547,34,3,714,3455,3714,1647,...,10818,5: Western Australia,15710.0,GLENDALOUGH,-31.921222,115.831181,-31.919233,115.820029,8.060640,
7,6018,21115,8872,2097,108,62,1674,6441,17255,9825,...,42250,5: Western Australia,15715.0,CHURCHLANDS,-31.891957,115.786821,-31.921546,115.789762,7.995266,
8,6019,8914,8379,2624,0,9,1096,7347,11623,4351,...,24351,5: Western Australia,15722.0,SCARBOROUGH,-31.907330,115.769183,-31.898032,115.771597,7.432960,
9,6020,12637,2698,819,0,87,361,2178,5389,7714,...,22581,5: Western Australia,15724.0,CARINE,-31.852698,115.765396,-31.851149,115.777646,6.213188,


In [52]:
# Testing Autocomplete

# Set the hotel search radius

search_type = "gym"
search_radius = 10000

search_loc = "" 

count = 0

params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
    "postal_code":  
}

# use iterrows to iterate through pandas dataframe
for index, row in wa_pop_greater_10000_df.iterrows():
    # get location from df
    search_postcode = row['Postcode']
 
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    count=count+1
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    gym_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"{count} - A Gym in {search_postcode} to try is {gym_name['results'][0]['name']}.")
        wa_pop_greater_10000_df.loc[index, "Gym"] = gym_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        wa_pop_greater_10000_df.loc[index, "Gym"] = "N/A"
        print("No results... skipping.")
        
    #print("----------------------------")

SyntaxError: invalid syntax (2113194059.py, line 17)

In [53]:
# Set the hotel search radius

search_type = "gym"
search_radius = 10000

search_loc = "" 

count = 0

params = {
    "types": search_type,
    "radius": search_radius,
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in wa_pop_greater_10000_df.iterrows():
    # get location from df
    search_postcode = row['Postcode']
 
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    count=count+1
    
    #change location each iteration while leaving original params in place
    #params["location"] = f"{lat},{lng}"
    
    search_loc = f"{lat},{lng}"
    params["location"] = search_loc
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" #moved
    search_result = requests.get(base_url, params=params)
    gym_name = search_result.json()
    
    #Store the first hotel in dataframe
    
    try:
    # Print the name and address of the first hotel to appear
        print(f"{count} - A Gym in {search_postcode} to try is {gym_name['results'][0]['name']}.")
        wa_pop_greater_10000_df.loc[index, "Gym"] = gym_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        wa_pop_greater_10000_df.loc[index, "Gym"] = "N/A"
        print("No results... skipping.")
        
    #print("----------------------------")

KeyError: 'Latitude'

In [54]:
gym_name["results"]

NameError: name 'gym_name' is not defined